# Trabajo entregable: Bot de Telegram  

Trabajo realizado por:  
Olayo, Miguel, Ángel y Arturo.

Como parte de su evaluación de la asignatura de **Ciencia de datos en Economía**, del **Máster de Ciencia de Datos (UV)**.   

_4 de mayo de 2022_

Hay dos opciones muy extendidas para hacer un bot de telegram en python, estas son las APIs:  

* [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI)  
* [python-telegram-bot](https://github.com/python-telegram-bot/python-telegram-bot)  

Nosotros hemos procedido con la primera porque es de la que encontramos una mejor documentación. Concretamente, un tutorial especialmente útil fue el de este [link](https://www.youtube.com/watch?v=NwBWW8cNCP4&t=355s&ab_channel=CSDojo), del canal **CSDojo**.  

La funcionalidad completa de esta API no la vamos a explicar, pero si que voy a listar de forma rápida como es el flujo de trabajo para la creación y puesta en marcha del BOT.  


1. Primero "registramos" bot en Telegram, mediante el llamado **BotFather**, que no es sino un bot que nos ayuda a meter nuestro bot en el sistema. Este bot nos pregunta el nombre que le queremos dar, y nos propordiona una **KEY** o **TOKEN** que nos dará acceso a nuestro bot via código.  
2. Ahora pasamos al código:  
    a. Instanciamos un objeto **bot**, el cual inicia sesión gracias a que tenemos un **TOKEN** identificador.  
    b. Se programan las funcionalidades mediante las opciones que nos da la API, de forma que establecemos una interación entre los mensajes que manda el usuario y las respuestas y acciones nuestras.   
    c. Por último, para que el bot se esté ejecutando y esperando por inputs de los usuarios, debemos de llamar un método concreto (`bot.polling()`) el cual hace justo eso, un bucle infinito que espera inputs y los procesa como hemos indicado. 

A continuación mostramos el código de nuestro bot. Su funcionalidad es la de `El juego del Ahorcado`, donde tienes que adivinar una palabra y tienes x intententos para proponer letra, y otros x intentos para adivinar la palabra oculta.  
Como no queríamos que el bot fuera _offline_, sino que queríamos mostrar como se podría interaccionar con la red para extraer información y darsela al usuario mediante telegram. Asi pues, optamos por obtener la lista de palabras con las que se juega al ahorcado, mediante técnicas de _web scrapping_, quedando así patente como sería el proceso, y tal como lo hemos hecho nosotros para una lista de palabras, se podría hacer para un website de noticias etc...

In [1]:
# importamos librerias  necesarias
from bs4 import BeautifulSoup
import requests
import numpy as np 
import matplotlib.pyplot as plt 
import unidecode
import re 
import telebot 
import os 


In [2]:
#los mensajes que representan el estado del juego
HANGMANPICS = ['''
  +------+
  |      |
         |
         |
         |
         |
=========''', '''
  +------+
  |      |
 O     |
         |
         |
         |
=========''', '''
  +------+
  |      |
 O     |
  |      |
         |
         |
=========''', '''
  +------+
  |      |
 O     |
 /|     |
         |
         |
=========''', '''
  +------+
  |      |
 O     |
 /|\    |
         |
         |
=========''', '''
  +------+
  |      |
 O     |
 /|\    |
 /       |
         |
=========''', '''
  +------+
  |      |
 O     |
 /|\    |
 / \     |
         |
=========''']


In [3]:
#el diccionario de palabras lo vamos a sacar de 
#https://es.wiktionary.org/wiki/Ap%C3%A9ndice:1000_palabras_b%C3%A1sicas_en_espa%C3%B1ol  
#con webscraping

#esta es la descarga "cruda"
url="https://es.wiktionary.org/wiki/Ap%C3%A9ndice:1000_palabras_b%C3%A1sicas_en_espa%C3%B1ol  "
r=requests.get(url)
soup=BeautifulSoup(r.content,parser="html.parser")



In [4]:
#y este es el procesado para quedarnos con palabras de entre 5 y 7 letras, para que no tengan la ñ lo cual 
#puede dar problemas porque trabajamos sin acentos por simplicidad tanto de quien juega, como del código

lis=soup.find_all("li",class_="")
palabras=[]
max_letras=7
min_letras=5
for i in lis:
    try:
        if (len(i.a.text)>=min_letras and len(i.a.text)<=max_letras and len(i.a.text.split())==1):
            palabras.append(i.a.text)
        else:
            pass
    except:
        pass
#quito las ultimas 4 porque parecen un eror o algo
palabras=palabras[:-4]
palabras=[ i for i in palabras if "ñ" not in i]
palabras=[unidecode.unidecode(i).lower() for i in palabras]



['humano',
 'persona',
 'gente',
 'hombre',
 'mujer',
 'adulto',
 'anciano',
 'senor',
 'cuerpo',
 'pierna',
 'talon',
 'rodilla',
 'muslo',
 'cabeza',
 'labio',
 'diente',
 'nariz',
 'barba',
 'bigote',
 'cabello',
 'oreja',
 'cerebro',
 'brazo',
 'hombro',
 'muneca',
 'palma',
 'trasero',
 'abdomen',
 'higado',
 'musculo',
 'cuello',
 'corazon',
 'mente',
 'pecho',
 'cintura',
 'cadera',
 'espalda',
 'sangre',
 'carne',
 'hueso',
 'gripe',
 'diarrea',
 'salud',
 'familia',
 'amigo',
 'colega',
 'pareja',
 'esposo',
 'padre',
 'madre',
 'hermano',
 'abuelo',
 'nieto',
 'primo',
 'sobrino',
 'especie',
 'muerte',
 'campo',
 'bosque',
 'selva',
 'costa',
 'playa',
 'laguna',
 'cerro',
 'energia',
 'animal',
 'perro',
 'cerdo',
 'caballo',
 'oveja',
 'raton',
 'tigre',
 'conejo',
 'dragon',
 'ciervo',
 'jirafa',
 'pajaro',
 'gallina',
 'gorrion',
 'cuervo',
 'aguila',
 'halcon',
 'camaron',
 'sardina',
 'calamar',
 'pulpo',
 'insecto',
 'bicho',
 'polilla',
 'arana',
 'mosca',
 'caracol'

In [6]:
len(palabras)

623

In [7]:
#luego hacemos las funciones necesarias para que nos completen los huevos de letras 

def complete_letters(letters,word):
    word=list(word)
    cuales=[True if i in letters else False for i in word]
    final=[word[i] if cuales[i] else "_ " for i in range(len(cuales))]
    return "".join(final)

In [11]:
#asi funciona
complete_letters(["a","r"],"arturo")

'ar_ _ r_ '

La celda siguiente es la que declara todas las funciones que manejan la interacción con el usuario. Entender los flujos de mensajes en esta celda es un poco complejo si no se va paso a paso, pero a _groso modo_ se puede entender como:  

1. Entramos a jugar y la variable **jugando** es `False`.  
2. Le pedimos empezar con `\start`, y la variable **jugando** cambia a `True`, también se escoge una palabra aleatoria, y se inicializan las variables de "estado" del jugador, como el número de intentos.  
3. Hay dos funciones que comprueban si lo que se intruduce es una letra `enter_oneletter()`, en cuyo caso se realizan unas acciones, o una palabra `enter_word()`, en cuyo caso se hacen otras acciones.  
4. A lo largo del proceso se hacen comprobaciones sobre las vidas del jugador, la validez de los intentos etc.  
5. Finalmente, al ganar o perder, la variable **jugando** cambia a `False`, y estamos en las mismas, empezando el juego con todo inicializado de nuevo.  

_Nota: A mitad del juego siempre se puede reiniciar la adivinación de una nueva palabra, mediante `/start`._

In [35]:
#el bot tiene que tener diferentes stages, y debe ir evolucionando conforme se juega
#y además saber cuando se termina 
KEY= #aqui va el código obtenido del BtoFather

#se instancia el bot y se hace login
bot = telebot.TeleBot(KEY, parse_mode=None) # You can set parse_mode by default. HTML or MARKDOWN

#vamos a tener muchas callbacks pero solo se activaran cuando se le indique


#luego al adivinar se deben de pasar las letras una a una 
#pero si se pasa algo más largo se debe de detectar la palabra "solucion" o "respuesta" para proceder.

palabra=palabras[np.random.randint(0,len(palabras))]
jugando=False
letras=[]
hang_count=0
intentos_adiv=0
intentos_adiv_max=3


#primero debemos empezar
@bot.message_handler(commands=["start"])
def empezar(message):
    global jugando
    global palabra
    global palabras
    global letras
    global hang_count
    global intentos_adiv
    global intentos_adiv_max

    if jugando == False:
        jugando=True
        bot.send_message(message.chat.id, "Este es el juego del ahorcado.\nPara pedir letra simplemente escribela en el chat. \nY para adivinar la palabra pon: 'Respuesta: {la palabra}'")
        bot.send_message(message.chat.id,f"Y tienes {intentos_adiv_max} intentos.")
    elif jugando == True:
        bot.send_message(message.chat.id, "Reiniciamos el juego")
    letras=[]
    hang_count=0
    intentos_adiv=0
    palabra=palabras[np.random.randint(0,len(palabras))]
    print(letras,palabra,complete_letters(letras,palabra))
    bot.send_message(message.chat.id, f"La palabra a adivinar es: \n {complete_letters(letras,palabra)}")
    bot.send_message(message.chat.id, f"{HANGMANPICS[0]}")





def enter_oneletter(message):
    x=message.text
    #funcion que activa la respueta si estamos jungando y se ha enviado una letra
    global jugando
    if jugando and len(x.strip())==1 and x.strip().isalpha():
        return True
    elif jugando and len(x.strip())==1 and not x.strip().isalpha():
        bot.send_message(message.chat.id, "kè¿?!")
        return False 
    else:
    	#bot.send_message(message.chat.id, "kè¿?!")
        return False

@bot.message_handler(func=enter_oneletter)
def oneletter(message):
    x=message.text
    global palabra
    global letras
    global hang_count
    #entra si le he pasado una letra y estamos jugando

    #comprobamos si esta la letra en la palabra y sino sumamos una al moñeco
    if x.strip().lower() in letras:
        bot.send_message(message.chat.id, "Esa letra ya la ha dicho. Di otra venga.")

    elif x.strip().lower() in list(palabra):
        #si esta entonces palante y no sumamos
        letras.append(unidecode.unidecode((x.strip().lower())))
        
        bot.send_message(message.chat.id, f"La letra sí está, la palabra queda así: \n {complete_letters(letras,palabra)}")
        print(letras,palabra,complete_letters(letras,palabra))

    else:
        letras.append(unidecode.unidecode((x.strip().lower())))

        #si no esta sumamos una y printeamos
        hang_count+=1
        bot.send_message(message.chat.id, f"Esa letra NO está en la palabra: \n {complete_letters(letras,palabra)}")

        #comprobacion de que no hemos llegado al final de intentos
        bot.send_message(message.chat.id, f"{HANGMANPICS[hang_count]}")

        if hang_count>=6:
            #ha perdido
            jugando=False
            bot.send_message(message.chat.id,f"Has perdido, escribe '/start' para empezar una nueva partida.La palabra era '{palabra}'.")
            


#y si nos dice la palabra pues pasamo con eso a ver si la acierta
def enter_word(message):
    x=message.text
    global palabra
    global jugando
    #si metemos Respuesta: palabra
    #entonces comprobamos
    regex_aux=re.findall("^(.+):",x)
    if jugando and regex_aux:
        start_aux=unidecode.unidecode((regex_aux[0].lower().strip()))
        try:
            intento_aux=re.findall(":(.+)$",x)[0]
            palabra_intento=unidecode.unidecode(intento_aux.lower().strip())
        except:
            palabra_intento=None
        if jugando and len(x.strip())!=1 and (start_aux=="respuesta" or start_aux=="solucion") and palabra_intento is not None:
            return True
        elif jugando and len(x.strip())!=1 and (start_aux!="respuesta" and start_aux!="solucion"):
            bot.send_message(message.chat.id, "kè¿?!") #no se entiende el input
            return False    
        else:
            return False
    elif jugando and len(x.strip())!=1 :
        bot.send_message(message.chat.id, "kè¿?!") #no se entiende el input
        return False
    else:
        return False
    
@bot.message_handler(func=enter_word)
def word(message):
    x=message.text
    global palabra
    global intentos_adiv
    global jugando
    #comprobamos si la palabra es
    palabra_intento=re.findall(":(.+)$",x)[0].lower().strip()
    if palabra_intento == palabra:
        #si es, terminamos y todo guay, se reinicia todo y damos la oportunidad de volver a empezar
        bot.send_message(message.chat.id, "Muy bien, la has adivinado, si quieres jugar otra vez pon '/start'")
        jugando=False

    else:
        #si no es, sumamos uno a lo de los intentos y le avisamo que no es.
        intentos_adiv+=1
        bot.send_message(message.chat.id, f"NO! ERROR, MAL!!, {palabra_intento} NO ES LA PALABRA.")
        
        #simplemente para poner el plural al mensaje de los intentos
        if (intentos_adiv_max-intentos_adiv)>1:
            plural1="n"
            plural2="s"
        else:
            plural1=""
            plural2=""

        bot.send_message(message.chat.id, f"Te queda{plural1} {intentos_adiv_max-intentos_adiv} intento{plural2} para adivinar la palabra.")
        #si no la a avinado y ya estamos en el máximo le avisamo y volvemos a comentar
        if intentos_adiv>=intentos_adiv_max:
            #terminamos
            jugando=False
            bot.send_message(message.chat.id, "Has agotado los intentos para adivinar la palabra, pon '/start' para reiniciar el juego.")

        

#tenemos que tener una funcion que compruebe que no se han pasado los intentos de letras y de palabras 




In [37]:
#por último se ejecuta el bot para que quede a la espera de mensajes
bot.polling()

[] nacer _ _ _ _ _ 


_Si necesitara una explicación detallada del código, con mucho gusto podemos hacer una reunion_